In [1]:
# DEPS: 

!pip install pandas numpy nltk joblib scikit-learn tensorflow snscrape

You should consider upgrading via the 'C:\Users\Gustavo\Documents\PythonProjects\twitter-toxicity\venv\Scripts\python.exe -m pip install --upgrade pip' command.


# Load data

In [2]:
data_folder = '../data/'

In [3]:
import zipfile
with zipfile.ZipFile(data_folder+'twittes_data.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [4]:
import pandas as pd

df = pd.read_csv('comentarios_toxicos_ptBR.csv')

df

,Unnamed: 0,text,text_norm,toxic
0,12451,brocolis é muito boooom pqp,brócolis boom puta pariu,0
1,4080,to correndo disso,to correndo disso,0
2,5195,presidente da oab sofre amea莽as de morte e aci...,presidente aba sofre ame-a morte aciona polega...,0
3,13375,@user @user aí desanima pq para de vir teus ch...,aí desanima porque vir champ chato pra caralho...,1
4,4130,"ser feio e beta é um bosta, puta que me pariu....",feio beta bosta puta pariu lá vou comprar ócul...,1
...,...,...,...,...
29917,1130,JOao Ostral um idiota assumido Que beleza,joão rostral idiota assumido beleza,1
29918,5390,kaillane fdp brt na minha casa pra n贸s joga um...,allan filho puta bus casa pra joga vi game 馃檮馃檮,1
29919,860,@user engraçado que milo tem 44 acho e o patti...,engraçado milho acho evans uns,0
29920,15795,Eu quero ver se vamos precisar ir para a série...,quero ver vamos precisar ir série demitir trei...,1


# Analyzing the data

In [5]:
print('Não Tóxicos: ', len(df[df['toxic'] == 0]))
print('Tóxicos: ', len(df[df['toxic'] == 1]))

Não Tóxicos:  16412
Tóxicos:  13510


# Preprocessing

In [6]:
import re 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('floresta')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
import joblib
import sys  
sys.path.insert(1, data_folder)
from abbreviations_synonyms import abbreviations_synonyms_dict

tagger = joblib.load(data_folder+'POS_tagger_bigram.pkl') # https://github.com/inoueMashuu/POS-tagger-portuguese-nltk/tree/master/trained_POS_taggers

def clean_text(text):
  text = ' '.join([ word for word in text.split(' ') if not word.startswith('@') ])
  text = re.sub(r"[^A-Za-z ]+", '', text) # keep only letters and spaces
  text = text.strip()
  return text

def replace_synonyms_abbreviations(text):
  for abbr_or_syn, full_text in abbreviations_synonyms_dict.items():
    text = re.sub(rf"\b{abbr_or_syn}\b",full_text,text)

  return text

def remove_stop_words(text):
  stopwords_pt = stopwords.words('portuguese')
  
  text_without_sw = [word for word in text.split(' ') if not word in stopwords_pt]
  return (" ").join(text_without_sw)

def lemmatization_nltk(text):
  lemmatizer = WordNetLemmatizer()
  palavras = nltk.word_tokenize(text, language='portuguese')
  lemmas = [lemmatizer.lemmatize(p).lower() for p in palavras]
  return (" ").join(lemmas)

def remove_proper_nouns(text):
  words = []
  for word,tag in tagger.tag(word_tokenize(text)):
    if tag != 'NPROP':
      words.append(word)
  return ' '.join(words)

def normalize_text(text):
  text = clean_text(text)
  text = remove_proper_nouns(text)
  text = text.lower() # outside clean_text because capitalization influences remove_proper_nouns function 
  text = replace_synonyms_abbreviations(text)
  text = remove_stop_words(text)
  text = lemmatization_nltk(text)
  return text


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package floresta to
[nltk_data]     C:\Users\Gustavo\AppData\Roaming\nltk_data...
[nltk_data]   Package floresta is already up-to-date!


In [7]:
# our own normalization
df = df[df['text'].notna()] # removing nan values
df['text_norm2'] = df['text'].apply(normalize_text)

C:\Users\Gustavo\AppData\Local\Temp\ipykernel_11812\2210283727.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_norm2'] = df['text'].apply(normalize_text)


In [8]:
df.head(50)

,Unnamed: 0,text,text_norm,toxic,text_norm2
0,12451,brocolis é muito boooom pqp,brócolis boom puta pariu,0,puta pariu
1,4080,to correndo disso,to correndo disso,0,correndo disso
2,5195,presidente da oab sofre amea莽as de morte e aci...,presidente aba sofre ame-a morte aciona polega...,0,presidente oab sofre ameaas morte aciona polci...
3,13375,@user @user aí desanima pq para de vir teus ch...,aí desanima porque vir champ chato pra caralho...,1,desanima porque vir chato pra caralho trocar c...
4,4130,"ser feio e beta é um bosta, puta que me pariu....",feio beta bosta puta pariu lá vou comprar ócul...,1,feio beta bosta puta pariu l vou comprar grau ...
5,18046,pra que?,pra,0,pra
6,6522,"rt @user nunca tem nada lá , e quando tem e pe...",nunca nada lá pergunta inglês,0,rt nunca nada l pergunta ingls
7,16147,amas aqui o arroz é seco filho da puta,amas aqui arroz seco filho puta,1,amas aqui arroz seco filho puta
8,10339,@user setembro o pau quebra,setembro pau quebra,0,setembro caralho quebra
9,8757,rt @user a verdade é que a pessoa pode ser lix...,verdade pessoa pode imunda bonitinha vão pagar...,0,rt verdade pessoa pode imundo bonitinha vo pag...


# Training NN

In [9]:
# Import functions from sklearn library
from sklearn.model_selection import train_test_split

# Splitting the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2,random_state=16)
print("Train Data size:", len(train_data))
print("Test Data size", len(test_data))

Train Data size: 23936
Test Data size 5985


In [10]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tweets_column='text_norm2'

tokenizer.fit_on_texts(train_data[tweets_column])
word_index = tokenizer.word_index
print(word_index)

{'pra': 1, 'voc': 2, 's': 3, 'puta': 4, 'caralho': 5, 'rt': 6, 'vai': 7, 'j': 8, 'porra': 9, 'ano': 10, 'q': 11, 'ter': 12, 'dia': 13, 'porque': 14, 'pariu': 15, 'so': 16, 'fazer': 17, 't': 18, 'est': 19, 'tudo': 20, 'n': 21, 'vou': 22, 'ainda': 23, 'nada': 24, 'bem': 25, 'cara': 26, 'vida': 27, 'cu': 28, 'algum': 29, 'merda': 30, 'sempre': 31, 'coisa': 32, 'ficar': 33, 'tempo': 34, 'nunca': 35, 'assim': 36, 'pode': 37, 'hoje': 38, 'agora': 39, 'toda': 40, 'sei': 41, 'tambm': 42, 'sobre': 43, 'todo': 44, 'aqui': 45, 'gente': 46, 'mim': 47, 'coisas': 48, 'foda': 49, 'casa': 50, 'ver': 51, 'mundo': 52, 'algo': 53, 'pessoas': 54, 'vez': 55, 'dar': 56, 'todos': 57, 'vc': 58, 'bom': 59, 'amigo': 60, 'quer': 61, 'faz': 62, 'nao': 63, 'melhor': 64, 'h': 65, 'quero': 66, 'depsito': 67, 'homem': 68, 'p': 69, 'pessoa': 70, 'qualquer': 71, 'ento': 72, 'queria': 73, 'l': 74, 'ir': 75, 'mulher': 76, 'acho': 77, 'menos': 78, 'apenas': 79, 'boa': 80, 'poi': 81, 'sim': 82, 'filho': 83, 'mano': 84, 'ta

In [11]:
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

Vocabulary Size : 43535


In [12]:
from keras.utils import pad_sequences

# The tokens are converted into sequences and then passed to the pad_sequences() function
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data[tweets_column]), maxlen = 30)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data[tweets_column]), maxlen = 30) 

In [13]:
y_train = train_data['toxic']
y_test = test_data['toxic']

print(y_train.shape)
print(y_test.shape)

(23936,)
(5985,)


In [14]:
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding,Flatten

# model = Sequential()
# model.add(Embedding(vocab_size, output_dim=2, input_length=30))
# model.add(SimpleRNN(32,return_sequences=False))
# model.add(Dense(1, activation='sigmoid'))
model = Sequential()
model.add(Embedding(vocab_size, output_dim=2, input_length=30))
model.add(SimpleRNN(15,return_sequences=True))
model.add(SimpleRNN(15))
model.add(Dense(1,activation='sigmoid'))

model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 2)             87070     
                                                                 
 simple_rnn (SimpleRNN)      (None, 30, 15)            270       
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 15)                465       
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 87,821
Trainable params: 87,821
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train, y_train, epochs=3, validation_data=(x_test,y_test))

Epoch 1/3
748/748 [==============================] - 9s 8ms/step - loss: 0.6397 - acc: 0.6316 - val_loss: 0.6283 - val_acc: 0.6705
Epoch 2/3
748/748 [==============================] - 6s 8ms/step - loss: 0.4940 - acc: 0.7560 - val_loss: 0.4848 - val_acc: 0.7694
Epoch 3/3
748/748 [==============================] - 6s 8ms/step - loss: 0.3406 - acc: 0.8475 - val_loss: 0.5149 - val_acc: 0.7581


# Testing MODEL

In [16]:
import snscrape.modules.twitter as sntwitter

#recuperando os tweets de uma conta
username = "biologia_braba"
max_tweets = 25

tweets = []
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f"from:{username}").get_items()):
    if i >= max_tweets:
        break
    tweets.append(tweet)

content_tweets = [tweet.rawContent for tweet in tweets]
print(content_tweets)

['Se liga eu arrombado, eu tô em live na Twitch jogando Mine, entra aí pra ser xingado ao vivo.\n\nhttps://t.co/aVplki53WT', '@UTEROSEMFURlA Ba é abreviação de Baltazar.', '@satur_n0 Já falei dessa, esse é antigo', 'Contato comercial para parcerias ou publicidade por DM ou email \n\n✉️murilomiguelba@gmail.com', 'Até a próxima aula arrombados.', 'Vai achando que na biologia não tem referências a filmes. https://t.co/sCpcIRdcsN', 'O nome "Água viva beijamin Button" (não beijamin arrola) é uma referência a uma filme onde o protagonista nasce velho e a medida que o tempo passa ele vai ficando mais e mais novo', 'Namorar, é praticamente um animal que consegue voltar no tempo, me diz se isso não é foda pra caralho', 'Essa água viva pode se reverter novamente ao estado de pólipo, devido a condições desfavoráveis do ambiente, danos ao corpo, etc.', 'Se tu é uma mula e não sabe o que isso significa, imagina que tu tem 90 anos tudo fudido de osteoporose, é como se você fosse capaz de voltar a te

In [17]:
import numpy as np

# cálculo de toxicidade 
tweets_normalized = [normalize_text(tweet) for tweet in content_tweets]
tweets_tokens = pad_sequences(tokenizer.texts_to_sequences(tweets_normalized), maxlen = 30)  

y = model(tweets_tokens, training=False)

for i, raw_tweet in enumerate(tweets_normalized):
  print(raw_tweet)
  print(content_tweets[i])
  print(f"Toxidade: {float(y[i])*100}%")
  print('------------------------------------------')

print(f"Média de toxicidade do twitter: {np.mean(y)*100}")

liga arrombado t live jogando entra pra xingado vivohttpstcoavplkiwt
Se liga eu arrombado, eu tô em live na Twitch jogando Mine, entra aí pra ser xingado ao vivo.

https://t.co/aVplki53WT
Toxidade: 55.64842224121094%
------------------------------------------
ba
@UTEROSEMFURlA Ba é abreviação de Baltazar.
Toxidade: 15.678344666957855%
------------------------------------------
dessa antigo
@satur_n0 Já falei dessa, esse é antigo
Toxidade: 13.435991108417511%
------------------------------------------
contato comercial parcerias publicidade
Contato comercial para parcerias ou publicidade por DM ou email 

✉️murilomiguelba@gmail.com
Toxidade: 9.05875563621521%
------------------------------------------
at prxima aula arrombados
Até a próxima aula arrombados.
Toxidade: 7.8680120408535%
------------------------------------------
vai achando biologia referncias filmes httpstcoscpcirdcsn
Vai achando que na biologia não tem referências a filmes. https://t.co/sCpcIRdcsN
Toxidade: 40.8815771341

# Export model

In [18]:
import io
import json 

model.save(data_folder+'exported/twitter_toxicity_model')
tokenizer_json = tokenizer.to_json()
with io.open(data_folder+'exported/tokenizer.json', 'w', encoding='utf-8') as f:
  f.write(json.dumps(tokenizer_json, ensure_ascii=False))

INFO:tensorflow:Assets written to: ../data/exported/twitter_toxicity_model\assets


INFO:tensorflow:Assets written to: ../data/exported/twitter_toxicity_model\assets
